## Stage IAP

In [3]:
import numpy as np

# Cartesien <-> Spherique
def cartesian_to_spherical(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arctan2(z, np.sqrt(x**2 + y**2))
    phi = np.arctan2(y,x)
    return r, theta, phi

def spherical_to_cartesian(r, theta, phi):
    x = r*np.sin(theta)*np.cos(phi)
    y = r*np.sin(theta)*np.sin(phi)
    z = r*cos(theta)
    return x, y, z

# Projection stereographique
def sphere_to_plane(x_n, y_n, z_n):
    x = 2*x_n/(1 + z_n)
    y = 2*y_n/(1 + z_n)
    return x, y

def plane_to_sphere(x, y):
    x_n = 4*x/(4 + x**2 + y**2)
    y_n = 4*y/(4 + x**2 + y**2)
    z_n = (4 - x**2 - y**2)/(4 + x**2 + y**2)
    return x_n, y_n, z_n

# Angle <-> Longueur
def f_to_length(angle_degrees):
    angle_radians = np.radians((180-angle_degrees)/2)
    x, y = sphere_to_plane(np.cos(angle_radians), 0, np.sin(angle_radians))
    return 2*x

def length_to_f(length):
    plane_coords = to_sphere(length/2, 0)
    alpha_radians = np.arctan2(plane_coords[2], plane_coords[0])
    alpha_degrees = np.degrees(alpha_radians)
    return 180 - 2*alpha_degrees

# Screen size
def screen_size(resX, resY, length):
    ratio = resX/resY
    width = length/ratio
    return length, width

# Screen <-> Planne
def plane_to_screen(resX, resY, x, y, f):
    L = f_to_length(f)
    X = (2*x/L)*resX + resX/2
    Y = (2*y/L)*resX + resY/2
    return X, Y

def screen_to_plane(resX, resY, X, Y, f):
    L = f_to_length(f)
    x = (X - resX/2)*L /(2*resX)
    y = (Y - resY/2)*L/(2*resX)
    return x, y

# Rotations
def R3(psi):
    return np.array([[np.cos(psi),   np.sin(psi),  0],
                     [-np.sin(psi),  np.cos(psi),  0],
                     [0,             0,            1]])

def R2(theta):
    return np.array([[1,             0,                0],
                     [0,  np.cos(theta),  -np.sin(theta)],
                     [0,  np.sin(theta),   np.cos(theta)]])

def R1(phi):
    return np.array([[np.cos(phi),   np.sin(phi),  0],
                     [-np.sin(phi),  np.cos(phi),  0],
                     [0,             0,            1]])

# Sun <-> Camera
def cam_to_sun(x_n, y_n, z_n, psi, theta, phi):
    vect_cam = np.array([x_n, y_n, z_n])
    vect_sun = vect_cam @ R3(psi) @ R2(theta) @ R1(phi)
    return tuple(vect_sun)

def sun_to_cam(u_n, v_n, w_n, psi, theta, phi):
    vect_sun = np.array([u_n, v_n, w_n])
    vect_cam = vect_sun @ (R3(psi) @ R2(theta) @ R1(phi)).T
    return tuple(vect_cam)

# Couleur associe a chaque position
def angle_to_colour(phi, theta):
    phi_tilde = phi // 5
    theta_tilde = theta // 5
    
    if (phi_tilde + theta_tilde) % 2 == 0:
        return 0, 0, 0
    return 255, 255, 255

# Cadrillage test rouge et bleu
def rouge_noir():
    # PPM header
    width = 256
    height = 128
    maxval = 255
    ppm_header = f"P3\n{width} {height}\n{maxval}\n"

    # PPM image data
    image = f""
    for i in range(height):
        # Replissage d'une ligne
        for j in range(width-1):
            if (j+i)%2:
                image = image + f"0 0 255 "
            else:
                image = image + f"255 0 0 "

        # Passage a la ligne
        if not (j+i)%2:
            image = image + f"0 0 255\n"
        else:
            image = image + f"255 0 0\n"

    # Save the PPM image as a binary file
    with open("rouge_noir.ppm", "wb") as f:
        f.write(bytearray(ppm_header, "ascii"))
        f.write(bytearray(image, "ascii"))

def pixel_to_colour(resX, resY, f, X, Y, psi, theta, phi):
    x, y = screen_to_plane(resX, resY, X, Y, f)
    x_n, y_n, z_n = plane_to_sphere(x, y)
    u_n, v_n, w_n = cam_to_sun(x_n, y_n, z_n, psi, theta, phi)
    r, theta, phi = cartesian_to_spherical(u_n, v_n, w_n)
    colour = angle_to_colour(phi, theta)
    return colour

pixel_to_colour(160, 90, 10, 10, 1, 0, 0, 0)

rouge_noir()